## LightGBM  (4) 
- player情報を付加
- 2017年の実績を付加
- 特徴量数: 128

In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lightgbm as lgb
import gc
%matplotlib inline
pd.set_option('display.max_Columns', 100)

In [27]:
ALL_MERGE = 'intermediate/all_merge_4.f'

In [28]:
all_pitch = pd.read_feather(ALL_MERGE)
all_pitch.shape

(778767, 143)

### 不要な列を削除

In [29]:
all_pitch.drop(
    columns=[
        '年度', 
        '試合ID', 
        'ホームチームID', 'アウェイチームID', 
        '投手ID', '投手チームID', 
        '打者ID', '打者チームID', 
        'プレイ前走者状況', 
        '捕手ID', 
        'opening_date', 'game_date',
        'start_time', 'game_time', 'elapsed_time'
    ], inplace=True)

### Rename

In [30]:
all_pitch.rename(columns={
    'データ内連番': 'No',
    '投球位置区域': 'pitch_area',
    '試合内連番': 'No_in_game',
    '試合内投球数': 'pitch_cnt_in_game',
    'イニング': 'inning',
    'イニング内打席数': 'bat_cnt_in_inning',
    '打席内投球数': 'pitch_cnt_in_bat',
    '投手登板順': 'pitch_order',
    '投手試合内対戦打者数': 'player_cnt_in_game',
    '投手試合内投球数': 'pitcher_cnt_in_game',
    '投手イニング内投球数': 'pitcher_cnt_in_inning',
    '打者打順': 'bat_order',
    '打者試合内打席数': 'bat_cnt_in_game',
    'プレイ前ホームチーム得点数': 'home_point',
    'プレイ前アウェイチーム得点数': 'away_point',
    'プレイ前アウト数': 'out_cnt',
    'プレイ前ボール数': 'ball_cnt',
    'プレイ前ストライク数': 'strike_cnt',
}, inplace=True)

In [31]:
print(all_pitch.shape)
all_pitch.head()

(778767, 128)


,No,ball,pitch_area,No_in_game,pitch_cnt_in_game,inning,bat_cnt_in_inning,pitch_cnt_in_bat,pitch_order,player_cnt_in_game,pitcher_cnt_in_game,pitcher_cnt_in_inning,bat_order,bat_cnt_in_game,home_point,away_point,out_cnt,ball_cnt,strike_cnt,first,second,third,base_cnt,pitch_LR,bat_LR,top_bot,role,pos_pit,date_from_opening,elapsed_min,home,point_diff,runner_23,cleanup,pinch,ball_cnt_0-0,ball_cnt_0-1,ball_cnt_0-2,ball_cnt_0-3,ball_cnt_1-0,ball_cnt_1-1,ball_cnt_1-2,ball_cnt_1-3,ball_cnt_2-0,ball_cnt_2-1,ball_cnt_2-2,ball_cnt_2-3,pit_bat_L_L,pit_bat_L_R,pit_bat_R_L,...,firm_bat,height_bat,weight_bat,draft_order_bat,salary_bat,batter_cnt_bat,bat_game_cnt_bat,foreigner_bat,company_bat,univ_bat,highsch_bat,age_bat,play_year_bat,salary_year_bat,salary_x_year_bat,bmi_bat,firm,height,weight,draft_order,salary,batter_cnt,bat_game_cnt,foreigner,company,univ,highsch,age,play_year,salary_year,salary_x_year,bmi,salary_dif_p-b,play_year_dif_p-b,age_dif_p-b,salary_year_dif_p-b,salary_x_year_dif_p-b,bmi_dif_p-b,salary_dif_p-c,play_year_dif_p-c,age_dif_p-c,salary_year_dif_p-c,salary_x_year_dif_p-c,bmi_dif_p-c,salary_dif_b-c,play_year_dif_b-c,age_dif_b-c,salary_year_dif_b-c,salary_x_year_dif_b-c,bmi_dif_b-c
0,0,0.0,0.0,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1,0,0,0.000000,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...,0,176,74,5.0,15000,523.0,119.0,0,1,1,0,32,8.0,1875.0,120000.0,23.889463,0,178,83,1.0,5000,443.0,135.0,0,1,1,0,28,4.0,1250.0,20000.0,26.196187,9000,-2.0,-3,2125.0,24000.0,2.141358,19000,2.0,1,2750.0,124000.0,-0.165367,10000,4.0,4,625.0,100000.0,-2.306725
1,1,0.0,8.0,2,2,1,1,2,1,1,2,2,1,1,0,0,0,0,1,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1,0,0,0.200000,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,...,0,176,74,5.0,15000,523.0,119.0,0,1,1,0,32,8.0,1875.0,120000.0,23.889463,0,178,83,1.0,5000,443.0,135.0,0,1,1,0,28,4.0,1250.0,20000.0,26.196187,9000,-2.0,-3,2125.0,24000.0,2.141358,19000,2.0,1,2750.0,124000.0,-0.165367,10000,4.0,4,625.0,100000.0,-2.306725
2,2,0.0,5.0,3,3,1,1,3,1,1,3,3,1,1,0,0,0,0,2,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1,0,0,0.383333,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,...,0,176,74,5.0,15000,523.0,119.0,0,1,1,0,32,8.0,1875.0,120000.0,23.889463,0,178,83,1.0,5000,443.0,135.0,0,1,1,0,28,4.0,1250.0,20000.0,26.196187,9000,-2.0,-3,2125.0,24000.0,2.141358,19000,2.0,1,2750.0,124000.0,-0.165367,10000,4.0,4,625.0,100000.0,-2.306725
3,3,0.0,12.0,4,4,1,2,1,1,2,4,4,2,1,0,0,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,0,0,1.016667,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,180,74,1.0,7040,268.0,81.0,0,0,0,1,40,22.0,320.0,154880.0,22.839506,0,178,83,1.0,5000,443.0,135.0,0,1,1,0,28,4.0,1250.0,20000.0,26.196187,16960,-16.0,-11,3680.0,-10880.0,3.191314,19000,2.0,1,2750.0,124000.0,-0.165367,2040,18.0,12,-930.0,134880.0,-3.356681
4,4,0.0,8.0,5,5,1,2,2,1,2,5,5,2,1,0,0,1,0,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,0,0,1.200000,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,180,74,1.0,7040,268.0,81.0,0,0,0,1,40,22.0,320.0,154880.0,22.839506,0,178,83,1.0,5000,443.0,135.0,0,1,1,0,28,4.0,1250.0,20000.0,26.196187,16960,-16.0,-11,3680.0,-10880.0,3.191314,19000,2.0,1,2750.0,124000.0,-0.165367,2040,18.0,12,-930.0,134880.0,-3.356681


### train
- 行数: 257117

In [32]:
train = all_pitch.dropna(subset=['pitch_area'])
train.shape

(257117, 128)

### test
- 行数: 521650

In [33]:
test = all_pitch[all_pitch['pitch_area'].isnull()]
test.shape

(521650, 128)

In [34]:
del all_pitch
gc.collect()

68

In [35]:
train_d = train.drop([
    'No', 
    'pitch_area', 
    'ball'
], axis=1)

In [36]:
test_d = test.drop([
    'No', 
    'pitch_area', 
    'ball'
], axis=1)

## Dataset作成

In [37]:
lgb_train = lgb.Dataset(train_d, train['ball'])

### LGBM学習
- multiclass : クラス数=8

In [38]:
%%time
lgb_param = {
        'objective' : 'multiclass',
        'boosting_type': 'gbdt',
        'metric' : 'multi_logloss',
        'num_class' : 8,
        'num_leaves' : 31,
        'seed' : 0,
        'learning_rate' : 0.1,
    }

# 学習
cv_results = lgb.cv(lgb_param, lgb_train,
                    num_boost_round=8000,
                    early_stopping_rounds=100,
                    verbose_eval=100,
                    nfold=4)

num_boost_round = len(cv_results['multi_logloss-mean'])
print('Best num_boost_round:', num_boost_round)
print('Best CV score:', cv_results['multi_logloss-mean'][-1])

[100]	cv_agg's multi_logloss: 1.1296 + 0.00184188
[200]	cv_agg's multi_logloss: 1.12083 + 0.00226441
[300]	cv_agg's multi_logloss: 1.11739 + 0.00257445
[400]	cv_agg's multi_logloss: 1.11596 + 0.00264362
[500]	cv_agg's multi_logloss: 1.11598 + 0.0026657
Best num_boost_round: 423
Best CV score: 1.1158111693845418
CPU times: user 56min 43s, sys: 36.8 s, total: 57min 20s
Wall time: 15min 18s


In [40]:
best_iter = int(num_boost_round * 1.1)

### 全体で再学習

In [41]:
%%time
lgb_model = lgb.train(lgb_param, lgb_train, num_boost_round=best_iter)

CPU times: user 15min 33s, sys: 7.62 s, total: 15min 40s
Wall time: 4min 10s


### Feature Importance

In [42]:
fi = lgb_model.feature_importance()
fn = lgb_model.feature_name()
df_feature_importance = pd.DataFrame({'feat_name':fn, 'feat_imp':fi})
df_feature_importance.sort_values('feat_imp', inplace=True)
df_feature_importance.tail(30)

,feat_name,feat_imp
123,salary_x_year_dif_b-c,1272
109,age_dif_p-b,1289
119,salary_dif_b-c,1340
6,player_cnt_in_game,1365
88,salary_year_bat,1397
107,salary_dif_p-b,1399
57,changeup_pit,1460
77,weight_bat,1481
55,shoot_pit,1515
90,bmi_bat,1555


In [43]:
%%time
predict = lgb_model.predict(test_d, num_iteration = lgb_model.best_iteration)

CPU times: user 6min 15s, sys: 2.14 s, total: 6min 17s
Wall time: 1min 40s


In [44]:
predict

array([[3.00412485e-01, 2.53668962e-02, 1.16832946e-01, ...,
        1.94920408e-07, 1.67011525e-06, 2.48056397e-01],
       [2.72730128e-01, 7.72864803e-02, 1.12722835e-01, ...,
        2.91695046e-07, 1.03410474e-06, 7.65917742e-02],
       [3.02160547e-01, 1.05596050e-01, 1.45404099e-01, ...,
        4.41866114e-07, 3.34577854e-06, 1.15802070e-01],
       ...,
       [5.31308054e-01, 6.25045518e-02, 5.52186406e-04, ...,
        1.14362385e-06, 1.66467519e-06, 1.04510294e-03],
       [5.65719468e-01, 3.06598974e-02, 3.96430443e-04, ...,
        1.31004324e-06, 1.58414632e-06, 6.78138576e-04],
       [5.39745378e-01, 3.39689962e-02, 4.56258808e-04, ...,
        1.37721404e-06, 1.11438629e-06, 8.73019275e-04]])

## 結果出力

In [45]:
submit = pd.DataFrame(predict)
submit.reset_index(inplace=True)
print(submit.shape)
submit.head()

(521650, 9)


,index,0,1,2,3,4,5,6,7
0,0,0.300412,0.025367,0.116833,0.306823,0.002506,1.949204e-07,1.670115e-06,0.248056
1,1,0.272730,0.077286,0.112723,0.453916,0.006752,2.916950e-07,1.034105e-06,0.076592
2,2,0.302161,0.105596,0.145404,0.248032,0.083002,4.418661e-07,3.345779e-06,0.115802
3,3,0.285217,0.092570,0.145143,0.346069,0.014067,3.077916e-07,1.756951e-06,0.116931
4,4,0.159693,0.037381,0.174471,0.475617,0.010498,2.426655e-07,7.250982e-07,0.142338


### 2017の平均
- ストレート           0.468254
- カーブ             0.074725
- スライダー           0.185806
- シュート            0.070633
- フォーク            0.083013
- チェンジアップ         0.054217
- シンカー            0.011361
- カットボール          0.051992

In [46]:
submit.mean()

index    260824.500000
0             0.493198
1             0.072326
2             0.187749
3             0.058084
4             0.078460
5             0.050025
6             0.008961
7             0.051198
dtype: float64

In [47]:
submit.to_csv('submit/lgbm_3.csv', header=False, index=False)

### 評価結果
#### 4
- CV score: 1.1158111693845418
- 評価結果  : 1.4299265

#### 3
- CV score: 1.1172290313076036
- 評価結果  : 1.4335806